In [1]:
import json
import pandas as pd
import time

from typing import Tuple

import s3fs
import tweepy

## Tweepy

In [2]:
with open('twitter_keys.txt') as json_file:
    keys = json.load(json_file)

In [3]:
API_KEY = keys["API_KEY"]
API_SECRET_KEY = keys["API_SECRET_KEY"]

ACCESS_TOKEN = keys["ACCESS_TOKEN"]
ACCESS_TOKEN_SECRET = keys["ACCESS_TOKEN_SECRET"]

In [4]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)

auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [5]:
api = tweepy.API(
    auth,
    wait_on_rate_limit=True,
    wait_on_rate_limit_notify=True
    #parser=tweepy.parsers.JSONParser()
)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [6]:
def create_tweet_df(most_recent_id=False, max_id=False) -> Tuple[pd.DataFrame, int]:
    
    tweet_tuples = []

    for tweet in tweepy.Cursor(
        api.search,
        q="*",
        lang="en",
        result_type="popular",
        #since_id=most_recent_id,
        #max_id=max_id,
        tweet_mode="extended",
        count = 100
    ).items():

        tweet_info = (
            tweet.full_text, 
            tweet.favorite_count, 
            tweet.retweet_count, 
            tweet.user.followers_count, 
            tweet.created_at,
            tweet.id
        )

        tweet_tuples.append(tweet_info)

    tweet_df = pd.DataFrame(
        tweet_tuples, 
        columns =[
            'Tweet', 
            'Num_Favorites', 
            'Num_Retweets', 
            'Num_Followers',
            'Created_At',
            'id'
        ])
    
    tweet_df.sort_values('Created_At', inplace = True)
    
    return tweet_df

In [7]:
#all_tweets_df = create_tweet_df()
all_tweets_df = pd.read_csv('tweet_df.csv').sort_values('Created_At')

most_recent_id = all_tweets_df['id'].iloc[-1]
max_id = all_tweets_df['id'].iloc[0]

In [8]:
all_tweets_df.shape[0]

584

This doesnt seem to be working... Not sure why. Need to look into that lol

In [9]:
max_tweets = 10000

while all_tweets_df.shape[0] < max_tweets:
    
    new_tweets_df = create_tweet_df(most_recent_id=most_recent_id)

    if new_tweets_df.shape[0] == 0:
        print("No new tweets; sleeping for 15 minutes. Current count: ", all_tweets_df.shape[0])
        time.sleep(15*60)
    else:
        most_recent_id = new_tweets_df['id'].iloc[-1]
        
        all_tweets_df.append(new_tweets_df)

Rate limit reached. Sleeping for: 681


KeyboardInterrupt: 

In [10]:
print("Length before duplicate drop: ", all_tweets_df.shape[0])

Length before duplicate drop:  584


In [11]:
all_tweets_df.drop_duplicates(inplace = True)

In [12]:
# Should be the same?...
print("Length after duplicate drop: ", all_tweets_df.shape[0])

Length after duplicate drop:  584


## AWS
`s3fs` plays ball with `pandas` behind the scenes, so provided you have generated your IAM User access keys, installed the `AWS CLI`, and run `aws configure`, you can just write to an existing S3 bucket like you would locally.

In [13]:
#tweet_df.to_csv("s3://twitterathena/tweet_df.csv", index=False)
all_tweets_df.to_csv("./tweet_df.csv", index=False)

In [ ]:
#test = pd.read_csv("s3://twitterathena/tweet_df.csv")

In [ ]:
#test